In [41]:
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm
import warnings
import json
import os

warnings.filterwarnings("ignore")

In [2]:
sector_tickers = 'B C Y P E F V I K U R'.split()
sector_tickers[-1] += 'E'
sector_tickers_list = ['XL' + x for x in sector_tickers]
market_ticker = 'SPX'
sector_tickers = ' '.join(sector_tickers_list)
sectors = yf.Tickers(sector_tickers)
market = yf.Ticker(market_ticker)

In [10]:
hist = sectors.history(start='2010-06-02', end='2019-01-01')
market_hist = market.history(start='2010-06-02', end='2019-01-01')
hist['Close'][sector_tickers_list]

[*********************100%***********************]  11 of 11 completed


,XLB,XLC,XLY,XLP,XLE,XLF,XLV,XLI,XLK,XLU,XLRE
Date,,,,,,,,,,,
2010-06-02,23.580242,NaN,27.815084,19.341745,35.748093,9.699415,23.726410,23.435608,18.272644,18.693918,NaN
2010-06-03,23.295855,NaN,27.908707,19.370676,36.229519,9.647058,23.864685,23.546120,18.456123,18.889460,NaN
2010-06-04,22.373560,NaN,26.827770,18.857113,34.954735,9.260916,23.165176,22.441040,17.864000,18.328905,NaN
2010-06-07,21.973890,NaN,26.317087,18.712448,34.649616,9.077658,23.132645,21.864820,17.605461,18.439718,NaN
2010-06-08,22.473471,NaN,26.538383,19.016241,35.225975,9.267457,23.189573,22.093719,17.638815,18.680885,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2018-12-24,44.390610,37.929352,89.065865,44.790520,44.753868,20.856806,76.173950,56.994274,55.635216,46.500328,26.937445
2018-12-26,46.378529,40.060867,94.343185,46.040573,47.538517,21.801020,79.496536,59.648460,58.995350,47.131634,27.795902
2018-12-27,47.231834,40.245785,94.846687,46.481773,47.787891,22.062780,80.374382,60.394657,59.420200,47.528458,27.859154


In [49]:
hist.columns
price_features = ['Open', 'High', 'Low', 'Close', 'Volume']
## Features to engineer: 
## NOTE: Everything below is calculated based on daily returns, not prices.
##     SMA: 5, 13, 21, 55, 243  ***DONE***
##     Volatility: std, ADR     ***DONE***
##     Financials: quareterly_financials
##     Temporal Features: DOW, Q   ***DONE***
##     Overall Market Features: SMA: 21, 144, 300  ***DONE***
##                              Vol: std: 13, 55   ***DONE***
##                              PC Ratio           ***DONE***
##                              Exchange Rates (EURO/USD, FRANC/USD, AUD/USD)
##                                             SMA: 8, 21
##                                             Vol: std - 8, 21, 89

In [22]:
def get_market_pcr_data(df=pd.read_csv('data/spxpc.csv')):
    df = df[['Date', 'SPX Put/Call Ratio']]
    df['Date'] = pd.to_datetime(df['Date'])
    df.rename(columns={'SPX Put/Call Ratio': 'pc_ratio'}, inplace=True)
    return df.set_index('Date')
get_market_pcr_data()

,pc_ratio
Date,
2010-07-06,1.91
2010-07-07,0.91
2010-07-08,1.12
2010-07-09,1.20
2010-07-12,1.49
...,...
2019-09-30,2.26
2019-10-01,2.07
2019-10-02,1.82


# Feature Engineering

In [48]:
def get_static_features(df, tickers):
    for ticker in tickers:
        df['Returns', ticker] = 1 + (df['Close', ticker] - 
                                     df['Open', ticker]) / df['Open', ticker]
    return df

def get_rolling_features(df, tickers, periods=[5, 13, 21, 55, 243]):
    for ticker in tqdm(tickers):
        for per in periods:
            df[f'{per}_SMA', ticker] = df['Returns', ticker].rolling(window=per).mean()
            df[f'{per}_vol', ticker] = df['Returns', ticker].rolling(window=per).std()
    return df

def get_market_data(df):
    df = df.copy()
    df['market_returns'] = 1 + (df['Close'] - df['Open']) / df['Open']
    for per in [21, 144, 300]:
        df[f'market_{per}_SMA'] = df['market_returns'].rolling(window=per).mean()
    for per in [13, 55]:
        df[f'market_{per}_vol'] = df['market_returns'].rolling(window=per).std()
    pcr_df = get_market_pcr_data()
    df = pcr_df.merge(df, on='Date')
    return df

def get_temporal_features(df):
    dates = pd.Series(df.index)
    df['DOW'] = dates.dt.dayofweek.values
    df['quarter'] = dates.dt.month.values // 4
    return df
    

df = get_static_features(hist, sector_tickers_list)
df = get_rolling_features(df, sector_tickers_list)
market_df = get_market_data(market_hist)
df = df.merge(market_df, on='Date')
df = get_temporal_features(df)

100%|██████████| 11/11 [00:00<00:00, 470.65it/s]


In [47]:
df

,"(Close, XLB)","(Close, XLC)","(Close, XLE)","(Close, XLF)","(Close, XLI)","(Close, XLK)","(Close, XLP)","(Close, XLRE)","(Close, XLU)","(Close, XLV)",...,Dividends,Stock Splits,market_returns,market_21_SMA,market_144_SMA,market_300_SMA,market_13_vol,market_55_vol,DOW,quarter
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-08,22.912455,NaN,35.659058,9.385316,22.495901,17.810017,19.291111,NaN,19.497097,23.675323,...,0,0,0.909068,0.968382,NaN,NaN,0.034879,NaN,3,1
2010-07-12,23.151770,NaN,35.747604,9.529604,22.599028,17.985777,19.356682,NaN,19.694706,23.691677,...,0,0,1.000000,0.971784,NaN,NaN,0.034879,NaN,0,1
2010-07-13,23.777081,NaN,36.176750,9.765711,22.995632,18.245224,19.531523,NaN,19.760572,23.871529,...,0,0,0.900012,0.973820,NaN,NaN,0.039559,NaN,1,1
2010-07-14,23.699883,NaN,36.135876,9.693572,23.043228,18.354025,19.538809,NaN,19.767157,23.871529,...,0,0,1.000000,0.973820,NaN,NaN,0.038912,NaN,2,1
2010-07-15,23.676718,NaN,36.095013,9.673894,23.035295,18.412613,19.597086,NaN,19.898890,23.945095,...,0,0,1.000000,0.976996,NaN,NaN,0.038912,NaN,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-24,58.281620,NaN,62.679020,27.538294,73.826668,64.939484,52.120358,28.001007,43.784225,82.566666,...,0,0,1.000000,1.048944,1.010838,1.006762,0.101101,0.077790,2,0
2018-01-25,58.667774,NaN,62.186935,27.483236,73.900795,64.825256,52.147133,27.940022,44.464138,83.272919,...,0,0,1.000000,1.048944,1.010838,1.006762,0.098891,0.077790,3,0
2018-01-26,58.925209,NaN,62.525757,27.685116,74.753441,65.853180,52.397018,28.018433,44.490284,85.066422,...,0,0,1.000000,1.048944,1.009970,1.006762,0.098891,0.077790,4,0
